In [9]:
import ase
from ase.build import bulk
from ase.lattice.cubic import FaceCenteredCubic
from ase.calculators.lammpslib import LAMMPSlib
import chemiscope as chemi
import os
import numpy as np

## Make a XYZ file ##

In [10]:
# Initial Data Parameters:

Atom = 'Cu'
Structure = 'fcc'
n_defects = 1 # Number of defects to be generated (removed atoms)

# Lattice parameters:
r_lat0 = 3.60 # [Angstrom]
E_co = 3.38e5 # [Joules]
Bulk_modulus = 133 # [GPa]
# Results from DFT calculations:
epsilon = 6.567 # [Joules]
sigma = 2.334e-10 # [Meters]
cutoff_radius = 10.0 # [Angstrom]
D = 5.292e-20 # epsilon for morse potential
alpha = 1.329e+10 # A for morse potential
r0 = 2.885e-10 # r_min for morse potential

In [11]:
# Generate the ASE bulk data (unused):
# aseBulkData = bulk(
#     Atom, 
#     Structure, 
#     a=r_lat0,
#     cubic=True
# )

In [12]:
# Generate the ASE lattice data (used):
aseLatticeData = FaceCenteredCubic(
    directions=[[1,-1,0], [1,1,-2], [1,1,1]],
    size=(2,2,3),
    symbol='Cu',
    pbc=(1,1,0)
)

In [13]:
# Generate a directory for the atom:
if not os.path.exists(Atom):
    os.makedirs(Atom)
# inside the {Atom} directory, generate a directory for the structure:
if not os.path.exists(f'{Atom}/XYZ'):
    os.makedirs(f'{Atom}/XYZ')
if not os.path.exists(f'{Atom}/JSON'):
    os.makedirs(f'{Atom}/JSON')
    

# Write the ASE data to a XYZ file:
aseLatticeData.write(f'{Atom}/XYZ/{Atom}_{Structure}.xyz')

## Generate defects (to do) ##

In [14]:
# Duplicate the XYZ file
os.system(f'cp {Atom}/XYZ/{Atom}_{Structure}.xyz {Atom}/XYZ/{Atom}_{Structure}_defect.xyz')

# Generate random defects in the XYZ file:
n_atoms = len(aseLatticeData)
for i in range(n_defects):
    defect = np.random.randint(1, n_atoms) + 2
    # Remove the ith line of the XYZ file corresponding to the defect:
    with open(f'{Atom}/XYZ/{Atom}_{Structure}_defect.xyz', 'r') as f:
        lines = f.readlines()
    with open(f'{Atom}/XYZ/{Atom}_{Structure}_defect.xyz', 'w') as f:
        for j, line in enumerate(lines):
            if j == 0:
                f.write(f'{n_atoms - n_defects}\n')
            elif j != defect:
                f.write(line)
        
    print(f'Removed atom {defect} from {Atom}_{Structure}_defect.xyz')


Removed atom 40 from Cu_fcc_defect.xyz


## Calculate Potentials ##

In [15]:
frames = ase.io.read(f'{Atom}/XYZ/{Atom}_{Structure}.xyz', ':') + ase.io.read(f'{Atom}/XYZ/{Atom}_{Structure}_defect.xyz', ':')

properties = []
for i, frame in enumerate(frames):
    Charges = frame.get_atomic_numbers()

    cmds = [f"pair_style lj/cut {cutoff_radius}", f"pair_coeff * * {epsilon} {sigma}"]
    frame.calc = LAMMPSlib(lmpcmds=cmds, atom_types={'Cu':1}, log_file='log.lammps')
    # LennardJonesData = frame.get_potential_energies()
    LennardJonesDataGlobal = frame.get_potential_energy()

    cmds = [f"pair_style morse {cutoff_radius}", f"pair_coeff * * {D} {alpha} {r0}"]
    frame.calc = LAMMPSlib(lmpcmds=cmds, atom_types={'Cu':1}, log_file='log.lammps')
    # MorseData = frame.get_potential_energies()
    MorseDataGlobal = frame.get_potential_energy()

    properties.append({
        # "LennardJones": {
        #     "target": "atom",
        #     "values": LennardJonesData,
        #     "units": "J",
        # },
        "LennardJonesGlobal": {
            "target": "structure",
            "values": np.array([LennardJonesDataGlobal]),
            "units": "J",
        },
        "AtomicNumbers": { # To delete (here because Chemiscope requires at least two properties)
            "target": "atom",
            "values": Charges,
            "units": "J",
        },
        "AtomicNumbers2": { # To delete (here because Chemiscope requires at least two properties)
            "target": "atom",
            "values": Charges,
            "units": "J",
        },
        # "Morse": {
        #     "target": "atom",
        #     "values": MorseData,
        #     "units": "J",
        # },
        "MorseGlobal": {
            "target": "structure",
            "values": np.array([MorseDataGlobal]),
            "units": "J",
        },
    })

print(properties)

[{'LennardJonesGlobal': {'target': 'structure', 'values': array([-7.399562e-57]), 'units': 'J'}, 'AtomicNumbers': {'target': 'atom', 'values': array([29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29,
       29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29,
       29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29,
       29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29,
       29, 29, 29, 29]), 'units': 'J'}, 'AtomicNumbers2': {'target': 'atom', 'values': array([29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29,
       29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29,
       29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29,
       29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29,
       29, 29, 29, 29]), 'units': 'J'}, 'MorseGlobal': {'target': 'structure', 'values': array([0.]), 'units': 'J'}}, {'LennardJonesGlobal': {'target': 'structure', 'valu

In [16]:
path = f"{Atom}/JSON/{Atom}.json.gz"
DatasetInformation = {
    'name': Atom + ' ' + Structure,
    'description': 'Semester Project 1, atom ' + Atom + ' in ' + Structure + ' structure',
    'authors': [
        'Maxime Saillen',
    ],
    'references': [
        'https://doi.org/10.1016/j.commatsci.2022.111206',
    ],
}

for i in range(len(frames)):
    if i == 1:
        DatasetInformation['name'] += ' with ' + str(n_defects) + ' defects'
        DatasetInformation['description'] += ' with ' + str(n_defects) + ' defects'
        path = f"{Atom}/JSON/{Atom}_defect.json.gz"

    chemi.write_input(
        path=path,
        frames=frames[i], 
        meta=DatasetInformation, 
        properties=properties[i], 
        environments=chemi.all_atomic_environments(frames[i]), 
        settings=None, 
        shapes=None, 
        parameters=None
    )